# TP 7: Classification multi-classe

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn import svm

## Importer le jeu de données `digits`

In [ ]:
from sklearn import datasets

Charger le jeu de données `digits` via la fonction `datasets.load_digits` dans un nom `digits`. C'est un objet `Bunch` qui ressemble à un dictionnaire. Vous pouvez afficher les clefs d'indice avec `digits.keys()` et regardez la description de ce jeu de données en affichant par `print` le contenu associé à la clef `DESCR` (grâce à `digits['DESCR']` ou de façon équivalente via `digits.DESCR`).

In [ ]:
digits = datasets.load_digits()
print(digits.DESCR)

**Question** Nommez `X_digits` et `y_digits` les valeurs associées aux clefs d'indice `data` et `target` qui correspondent aux données et aux valeur associées (les classes ou cibles).

In [ ]:
X_digits = digits.data
y_digits = digits.target
y_digits, X_digits.shape

La clef "images" est une liste d'images que l'on peut afficher. Voici les 50 premières :

In [ ]:
nrows, ncols = 5, 10
fig, ax = plt.subplots(nrows, ncols, figsize=(10, 5))
for i in range(nrows*ncols):
    ax[i//ncols,i%ncols].imshow(digits.images[i], cmap=plt.cm.gray_r)

**Question**: Découpez l'échantillon en `X_train`, `y_train`, `X_test`, `y_test` en gardant les 100 derniers exemples pour le test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_digits, 
                                                    y_digits, 
                                                    test_size=100, 
                                                    random_state=1)

len(y_train)


Vérifiez que vous avez bien les dimensions suivantes pour ces objets : ((1697, 64), (100, 64), (1697,), (100,))

**Question** affichez les dimensions des matrices correspondant aux 4 échantillons.

In [ ]:
#Fait

## Perceptron multiclasse "Un-contre-Tous"

**Question**: Observez le nombre de classes et etudier leur distribution dans `train` et dans le `test`. 

In [ ]:
classes_list = np.unique(y_digits).astype(int)
Xi_mean = [np.mean(X_digits[y_digits == cls], axis=0) for cls in classes_list]
Xi_var = [np.var(X_train[y_train == cls], axis=0) for cls in classes_list]


Nous sommes donc face à un problème **multiclasse**, ou le nombre de classes est *k=10*. Une technique pour traiter ce type de problèmes consiste à le reduire en *k* problemes binaires. Specifiquement, on construit un classifieur binaire pour chaque classe: tous les exemples $x_i$ associes a cette classe recoivent une etiquette $+1$, alors que les exemples des autres classes sont etiquettes $-1$. Ceci correspond a un codage binaire de chaque classe: elle est associee a un vecteur $z$ de taille $k$, tel que $z_i = +1$ if $y_i = k$ et $z_i = -1$ autrement. 

Cette technique s'appelle le **one-versus-all**. 



### Codage binaire des labels

**Question**:  Concretement, on cree donc un vecteur binaire "cible" $y\in\{−1,+1\}^N$ (ou $N$ est le nombre d'exemples) que l'on associe a chaque classe (via un dictionnaire) pour chaque jeu de donnees `train` et `test`. 

In [ ]:
donnees=[y_train,y_test]
assoc=["y_train","y_test"]
dic={}
res={}
for d in donnees:
    for k in range (10):
        z=[]
        for i in range(len(d)):
            if k==d[i]:
                z=z+[1]
            else:
                z=z+[-1]
    
        dic[k]=z
    res[assoc[donnees.index(d)]]=dic
print(res)
    

**Question**: On entraine ensuite $10$ classifieurs binaires, dans ce cas-ci un perceptron, que l'on associe chacune des classes. Utilisez l'implementation du perceptron fournie dans `sklearn`. Fixez le nombre d'iterations maximum a $1$, $10$, puis $100$).  

In [39]:
perc=Perceptron(max_iter=1)
perc.fit(X_train,res["y_train"][0])
perc.predict(X_test)



### Prediction multiclasse

**Question**: On cherche a present a predire une des $k$ classes, en utilisant la regle de decision suivante, ou $\bf{w}_y$ est le vecteur de parametres appris avec le perceptron binaire pour la classe $y$:

$$
\hat{y} = \text{argmax}_{y\in\{1,\ldots,k\}} \bf{w}_y^{\top} \bf{x}
$$
Predisez les etiquettes multiclasses successivement sur les donnees `train` et `test`. 

### Evaluation 

**Question:** Calculer le taux d'erreur (cad, nombre de prediction correctes/nombre d'exemples) sur `train` et `test`. 

**Question**: Evaluer egalement votre modele de classification multi-classe en termes de **precision (P)**, **rappel (R)** et **F1-score (F1)**:

$$
P = \frac{TP}{TP+FP} \\
R = \frac{TP}{TP+FN}\\
F1 = 2 \frac{P\times R}{P+R}
$$
ou TP designent les vrais positifs, FP les faux positifs, et FN les faux negatifs. Ces mesures sont fournies dans `sklearn.metrics`. Utiliser l'option `average='macro'`.



**Question:** En vue de visualiser les erreurs faites pour le modele, calculer la **matrice de confusion**, egalement disponible dans `sklearn.matrix`. Analyser cette matrice. Quelles sont les erreurs les plus frequentes? Tentez d'expliquer. 